# Homework 3 - Find the perfect place to stay in Texas!

##### Work done by Musie Meressa Berhe, Ilaria Servadio, Omid Ghamiloo.

Show which packages we used:

In [1]:
import csv #for read the data
#import os
#import glob
import pandas as pd # for work with data frame
import itertools as it
from itertools import islice
import string #We use it later for removing punctuation

import json #JSON Library of Python can parse JSON string or file

# we import nltk library for preprocess the documents:
from nltk.corpus import words as nltk_words
from nltk.tokenize import sent_tokenize, word_tokenize,RegexpTokenizer 
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

## Step 1: Data

Importing Data with **read_csv()**

In [2]:
df = pd.read_csv('Airbnb_Texas_Rentals.csv')
df.head(n = 2) #head(n) return the first n rows

,Unnamed: 0,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,1,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...
1,2,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...


We delete the column called **'Unnamed: 0'** from our dataframe because the row index does not have to be imported into the tsv files.
The content of the file should be the following:

`average_rate_per_night \t  bedrooms_count \t city \t date_of_listing \t description \t latitude\t longitude \t title \t url`

In [3]:
df = df.drop('Unnamed: 0', axis = 1) 
df.head(n = 2)

,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...
1,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...


In [4]:
len(df) #corresponds to the number of file

18259

## Step 2: Create documents

We create a .tsv file(where each column is separated by the tab) for **each record** of the dataset.
The number of lines in our 'Airbnb_Texas_Rentals.csv' file is **18259**, we could see this thanks to    *len(df)*. Therefore, we create 18259 tsv file.

In [6]:
#Read the file and create a single tsv file for each record.  
for i in range(len(df)):
    file = open('Doc_i/doc_'+ str(i+1) 
              + '.tsv','w', newline = '', encoding = 'utf-8') 
    for j in range(9): # 9 is the number of columns
        file.write('%s\t' %df.iloc[i,j])
    file.close()

**Details:**
- We store all the documents into an empty folder called *'Doc_i'* and this folder is created especially for them;
- str(i+1) : **i** is the dataframe index but we increase it onefold because in thi way the numbering of the documents start from 1 and not from 0;
- The name of each file is *doc_i.tsv*.

## Step 3: Search Engine
As a first common step, we preprocess the documents by:
- **Removing stopwords:** if there are words that don’t really have any meaning to our text we can just remove them. This words have sense in our language but in data analysis they are useless;
- **Removing punctuation:** ! # $ % & \ ' () * + , - . / : ; < = > ? @ [ \\ ] ^ _ ` {|} ~
- **Stemming:** the idea is take from the word the root stem of the word. We do this because for a lot of items we are going to have different variations of word based on their stems or at least sort of fixes at the end but really the actual meaning of that word is unchanged;
- **Lemmatizing:** is a very similar operation to stemming but the end result is a real word, so, the end result will be generally some form of synonym or it might be the original word. So, you might end up with a very different word but it will be a similar word with the same meaning;
- **d = enchant.Dict("en_US"):** we accept only words that are part of the English dictionary so we delete all the words of other languages;
- **if not word.isdigit():** we do not take the numbers.

We know that lemmatization may havWe accept only words that are part of the English dictionary so we delete all the words of other languages.e been a risky choice. In fact, using lemmatizing the answer to the query is larger than when it is not used. We have chosen to do this operation because the result could be more satisfactory for the customer. We show our reasoning with an ***example***:

In English (and most languages), words appear in different forms. Look at these two sentences:

`A garden with cactus` -> lemmatizer.lemmatize("cactus")) return **cactus**

`A garden with cacti`  -> lemmatizer.lemmatize("cacti") return **cactus**

Both sentences talk about the noun cactus, but they are using different inflections. When working with text in a computer, it is helpful to know the base form of each word so that you know that both sentences are talking about the same concept. Otherwise the strings “cactus” and “cacti” look like two totally different words to a computer.


In [7]:
# Normalize the word. i.e. tokenize,stemming, lemmatization and removing punctuations
exclude = set(string.punctuation)
def wordNorm(text):
    stemmer = SnowballStemmer("english")
    lemmatizer = WordNetLemmatizer()
    Normalized = []
    tokenizer = RegexpTokenizer(r'\w+')
    word_list = tokenizer.tokenize(text.lower())
    Remove_stopwords = [stemmer.stem(lemmatizer.lemmatize(word)) for word in word_list if word not in stopwords.words('english')]
    for word in Remove_stopwords:
        word = ''.join(ch for ch in word if ch not in exclude) #remove the punctuation
        Normalized.append(word)
    return Normalized

### 3.1) Conjunctive query
At this moment, we narrow out interest on the **description** and **title** of each document.

Now, what we do is:

1) `Read` all the documents inside the folder which contains all the 18259 tsv files;

2) `Preprocess` the discription and title for each file;

3) Create a `list` of all the words "clean";

4) Create a `dictionary` whose key is the name of the document and the words contained in it for the items.

In [8]:
words = []
dict_ = {}
for i in range(len(df)):
    file = open('Doc_i/doc_'+ str(i+1) 
              + '.tsv','r', encoding = 'utf-8') 
    file_name = 'doc_'+ str(i+1) + '.tsv' 
    list_ = file.read().split('\t')
    desc_and_title = list_[4] + list_[7]
    cleaning = wordNorm(str(desc_and_title))
    words.append(cleaning)
    dict_[file_name] = cleaning
    file.close()
words = list(it.chain(*words)) # Convert a nested list into one-dimensional list

If you are interested in seeing the complete list of our words, open the file `'listwords.txt'` which is in the GitHub repository.

In [ ]:
# Lets save the words list for future use
with open('listwords.txt', 'w') as filehandle:  
    json.dump(words, filehandle)
    
#To read the listword and save it to our words variable (we impor json)
# open listwords file for reading
with open('listwords.txt', 'r') as filehandle:  
    words = json.load(filehandle)

**CreateVocabulary(words)**: this function take in input all words, create the set and assign an ID to each term.

*Vocabulary = {term : termID}*

In [10]:
def CreateVocabulary(words):
    # vocabulary = {term : termID}
    Vocabulary = {}
    wordList = list(set(words))
    for ID, element in enumerate(wordList):
        Vocabulary.update({element : ID})
    return Vocabulary

In [11]:
voc = CreateVocabulary(words)

If you are interested in seeing the id associated with each word, open the file `'vocabulary.txt'` which is in the GitHub repository.

In [13]:
# Lets save the vocabulary for future use
with open('vocabulary.txt', 'w') as filehandle:  
    json.dump(voc, filehandle)

The total number of words is:

In [14]:
len(voc)

13836

### 3.1.1) Create your index!
*invertedIndex = {termID : doc}*

In [15]:
#Build invertedIndex
# invertedIndex = {termID : doc}
def invertedIndex(vocabulary):
    invertedIndex= {}
    for key,value in vocabulary.items():
        for i in value:
            if (voc[i] in invertedIndex):
                invertedIndex[voc[i]] = invertedIndex[voc[i]] + [key]
            else:
                invertedIndex[voc[i]] = [key]
    return invertedIndex

In [16]:
invertedDic = invertedIndex(dict_)

If you are interested in seeing the inverted index, open the file `'inverted.json'` which is in the GitHub repository.

In [18]:
#Save our invertedIndex file for future use.
with open('inverted.json', 'w') as fp:
    json.dump(invertedDic, fp)

### 3.1.2) Execute the query
Given a query, that we let the user enter.

In [19]:
query = str(input().split())

beautiful house garden


In [20]:
qnorm = wordNorm(query) #Normalize the query

Show how the query became after applying the function `wordNorm(query)` which applies stemming, lemmatization and removing punctuations.

In [21]:
qnorm

['beauti', 'hous', 'garden']

**What documents do we want?**
Since we are dealing with conjunctive queries (***AND***), each of the returned documents should contain all the words in the query. 
`AND <--> intersection`

In [22]:
inter = set() #set with the documents that contain all the words in the query
switch = True
for word in qnorm:
    if word in voc:
        if (switch):
            inter = inter.union(invertedDic[voc[word]])
            switch = False
        if(len(inter) >0):
                inter = inter.intersection(set(invertedDic[voc[word]]))
        else:
            break

In [23]:
inter

{'doc_10416.tsv',
 'doc_10589.tsv',
 'doc_11332.tsv',
 'doc_11604.tsv',
 'doc_12060.tsv',
 'doc_12545.tsv',
 'doc_12928.tsv',
 'doc_13310.tsv',
 'doc_13696.tsv',
 'doc_14035.tsv',
 'doc_14255.tsv',
 'doc_14553.tsv',
 'doc_15036.tsv',
 'doc_15559.tsv',
 'doc_15718.tsv',
 'doc_16980.tsv',
 'doc_17121.tsv',
 'doc_17598.tsv',
 'doc_2.tsv',
 'doc_2057.tsv',
 'doc_2558.tsv',
 'doc_2824.tsv',
 'doc_3233.tsv',
 'doc_3385.tsv',
 'doc_3759.tsv',
 'doc_3877.tsv',
 'doc_4112.tsv',
 'doc_4207.tsv',
 'doc_5510.tsv',
 'doc_567.tsv',
 'doc_6226.tsv',
 'doc_6448.tsv',
 'doc_7279.tsv',
 'doc_7491.tsv',
 'doc_7928.tsv',
 'doc_8131.tsv',
 'doc_8481.tsv',
 'doc_8484.tsv',
 'doc_9138.tsv',
 'doc_9436.tsv',
 'doc_9948.tsv'}

### Final output of the query

In [28]:
my_dataframe = pd.DataFrame()
for file in inter:
    table = pd.read_table('Doc_i/'+ file,sep='\t', header = None,index_col = None)
    my_dataframe = my_dataframe.append(pd.DataFrame(table))

df1 = my_dataframe.rename(columns={0: 'average_rate_per_night', 1:'bedrooms_count', 2:'City',
                  3:'date_of_listing',4:'Description',5:'latitude',
                                                6:'longitude',7:'Title',8:'Url'})

df2 = df1[['Title','Description','City','Url']]

df2

,Title,Description,City,Url
0,East Austin Hillside Gem,"Beautiful and modern 3Br, 2.5Ba located minute...",Austin,https://www.airbnb.com/rooms/17555039?location...
0,Ensuite room in beautiful house near Uptown/DART,Private ground floor ensuite room in 4 storey ...,Dallas,https://www.airbnb.com/rooms/13262768?location...
0,Alla's Rose Room in S.W. Dallas,Beautifully furnished room with an antique Kin...,Duncanville,https://www.airbnb.com/rooms/714222?location=C...
0,Slade Street Carriage Pool House,This historic carriage house has been beautifu...,Coldspring,https://www.airbnb.com/rooms/15882909?location...
0,Serene Farmhouse on Natural Reserve,Welcome to Karakia - a beautiful farm house on...,Wimberley,https://www.airbnb.com/rooms/9509505?location=...
0,Rooms at Rehoboth,Step back into yesterday in this old Victorian...,Temple,https://www.airbnb.com/rooms/18055961?location...
0,Dallas Downton Abbey,"Our Dallas Downton Abbey is close to downtown,...",Dallas,https://www.airbnb.com/rooms/14254297?location...
0,The Vintage room in Fort Worth,Our place is a beautiful cozy open concept hou...,Fort Worth,https://www.airbnb.com/rooms/18959678?location...
0,"Home Away from Home in Spring, TX",Beautiful 2100 sq. ft house! Three bedroom and...,Spring,https://www.airbnb.com/rooms/1521227?location=...
0,The Vintage room in Fort Worth,Our place is a beautiful cozy open concept hou...,Fort Worth,https://www.airbnb.com/rooms/18959678?location...


### 3.2) Conjunctive query & Ranking score

In [ ]:
def term_freq(term, text):
    count = 0
    if len(text) <= 0:
        return 0
    else:
        for ter in text:
            if ter == term:
                count += 1
        return count / len(text)

In [ ]:
dic = 
for word in voc:
    tf = term_freq(word,voc)

In [ ]:
my_dataframe = pd.DataFrame()
for file in inter:
    print(file)